<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #C66363 ; color : #E8D6D8; text-align: center; border-radius: 100px 100px;">CONTENT </h1>
<br>

* [Add Libaries](#1)
* [Load and Examine Data](#2)
    * Examine Data
    * Handle Data
* [Create and Train Models](#3)
    * CNN Clasiffier
    * Visualize Result
    * CNN Output
    * SVM Clasiffier
    

<a id="1"> </a>
# Add Libaries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import torch.nn as nn 
import torch
import torch.nn.functional as f
import torch
from torch.autograd import Variable
import itertools
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<a id="2"> </a>
# Load and Examine Data

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train.head(3)

In [ ]:
y_train=train[["label"]]
x_train=train.loc[:,train.columns != "label"]

In [ ]:
x_train.head(3)

In [ ]:
y_train.head(3)

In [ ]:
ax = sns.countplot(x="label", data=y_train, palette="Set3")#Visualize the count of labels

In [ ]:
x_t=x_train.to_numpy().reshape((-1,28,28))
y_t=y_train.to_numpy().reshape((-1))
x_t.shape
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.imshow(x_t[i],cmap='gray')
  plt.title("value of frame:"+str(y_t[i]))
  plt.xticks([])

In [ ]:
Y_train = train.label.values

In [ ]:
X_train = x_train/255 #rescale image
X_test = test/255


In [ ]:
i_test=[i+1 for i in range(len(X_test))]

In [ ]:
X_test

In [ ]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train, test_size = 0.2, random_state = 42)

In [ ]:
X_train_s = X_train#For SVM Model
Y_train_s = Y_train

X_train = torch.Tensor(X_train)
X_val = torch.Tensor(X_val)
X_test = torch.Tensor(X_test)
i_test = torch.Tensor(i_test)
Y_train = torch.Tensor(Y_train).type(torch.LongTensor) 
Y_val = torch.Tensor(Y_val).type(torch.LongTensor) 

In [ ]:
X_test.shape

<a id="3"> </a>
# Create and Train Models

## CNN Clasiffier

In [ ]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(32 * 4 * 4, 10)
        
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # flatten
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out


In [ ]:
batch_size = 100
n_iters = 2500
num_epochs = n_iters / (len(X_train) / batch_size)
num_epochs = int(num_epochs)

trn = torch.utils.data.TensorDataset(X_train,Y_train)
val = torch.utils.data.TensorDataset(X_val,Y_val)
tst = torch.utils.data.TensorDataset(X_test,i_test)

train_loader = torch.utils.data.DataLoader(trn, batch_size = batch_size, shuffle = False)
val_loader = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(tst, batch_size = batch_size, shuffle = False)

model = CNN_Model()

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

error = nn.CrossEntropyLoss()

In [ ]:
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        train = Variable(images.view(100,1,28,28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train)
        
        loss = error(outputs, labels)
        
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 50 == 0:
                    
            correct = 0
            total = 0
            for images, labels in val_loader:
                
                val = Variable(images.view(100,1,28,28))
                
                outputs = model(val)
                
                predicted = torch.max(outputs.data, 1)[1]
                
                total += len(labels)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        if count % 500 == 0:
            # Print Loss
            print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data, accuracy))

In [ ]:
type(test_loader)

## Visualize Result 

In [ ]:
f = plt.figure(figsize=(20,6))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)

ax.plot(iteration_list,loss_list,color = "red")
ax.set_xlabel("Number of iteration")
ax.set_ylabel("Loss")
ax.set_title("CNN: Loss vs Number of iteration")

ax2.plot(iteration_list,accuracy_list,color = "green")
ax2.set_xlabel("Number of iteration")
ax2.set_ylabel("Accuracy")
ax2.set_title("CNN: Accuracy vs Number of iteration")


## Output 

In [ ]:
test_outputs=[]
for images, index in test_loader:
                
    val = Variable(torch.Tensor(np.array(images).reshape(100,1,28,28)))
                
    outputs = model(val)
                
    predicted = torch.max(outputs.data, 1)[1]
    
    predicted = predicted.tolist()
    
    test_outputs.append(predicted)


In [ ]:
test_outputs = list(itertools.chain.from_iterable(test_outputs))

In [ ]:
len(test_outputs)

In [ ]:
Label = pd.Series(test_outputs, name = "Label").astype(int)
ImageId =  pd.Series(i_test, name = "ImageId").astype(int)
results = pd.concat([ImageId, Label],axis = 1)
results.to_csv("digits.csv", index = False)

In [ ]:
results.head()

## SVM Clasiffier

In [ ]:
classifier = svm.SVC(gamma=0.001)
classifier.fit(X_train_s,Y_train_s)

In [ ]:
y_pred = classifier.predict(X_train_s)

cm=metrics.confusion_matrix(Y_train_s,y_pred)

In [ ]:
plt.figure(figsize=(15,8))
ax=sns.heatmap(cm/np.sum(cm), annot=True, 
            fmt='.2%')
ax.set(xlabel='Target', ylabel='Predicted Values',title="SVM CONFUSION MATRIX")
